<a href="https://colab.research.google.com/github/anandsuman2012/spark-notes/blob/master/PySpark_Crash_Course.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

In [32]:
mydata = spark.read.format("csv").option("header", "true").load("original.csv")

In [33]:
mydata.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52|39.9947462|116.3397725|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16|53.4266145| -6.1644997|
|  7|     Masha|    Divers|Female|         Dachun|     

In [34]:
from pyspark.sql.functions import *

In [83]:
mydata2 = mydata.withColumn("clean_city", when(mydata.City.isNull(),"Unknown").otherwise(mydata.City))

In [36]:
mydata2.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|     clean_city|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|      Nowa Ruda|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|         Bulgan|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52|39.9947462|116.3397725|        Unknown|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|  Divnomorskoye|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|      Mytishchi|
|  6|     Maris|      Folk|Femal

In [84]:
mydata2 = mydata2.filter(mydata.JobTitle.isNotNull())

In [38]:
mydata2.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|     clean_city|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|      Nowa Ruda|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|         Bulgan|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|  Divnomorskoye|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|      Mytishchi|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16|53.4266145| -6.1644997|Kinsealy-Drinan|
|  8|   Goddart|     Flear|  Mal

In [85]:
mydata2 = mydata2.withColumn(
    "clean_salary",
    mydata2.Salary.substr(expr("2"), length(mydata2.Salary) - 1 ).cast("float")
)


In [40]:
mydata2.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|     clean_city|clean_salary|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|      Nowa Ruda|    57438.18|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|         Bulgan|     62846.6|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|  Divnomorskoye|    61489.23|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|      Mytishchi|    63863.09|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil 

In [27]:
# Subtracting 1 from the length to get the length of the substring you want
mydata3 = mydata2.withColumn(
    "clean_salary",
    mydata2.Salary.substr(lit(2), length(mydata2.Salary) - lit(1)).cast("float")
)


In [28]:
mydata3.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|     clean_city|clean_salary|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|      Nowa Ruda|    57438.18|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|         Bulgan|     62846.6|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|  Divnomorskoye|    61489.23|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|      Mytishchi|    63863.09|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil 

In [86]:
mean_salary = mydata2.groupBy().avg("clean_salary").take(1)[0][0]

In [51]:
mean_salary

55516.32088199837

In [87]:
mydata2 = mydata2.withColumn("new_salary", round(when(mydata2.clean_salary.isNull(), lit(mean_salary)).otherwise(mydata2.clean_salary),2))

In [88]:
mydata2.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|     clean_city|clean_salary|new_salary|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+------------+----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|      Nowa Ruda|    57438.18|  57438.18|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|         Bulgan|     62846.6|   62846.6|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|  Divnomorskoye|    61489.23|  61489.23|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|      Mytishchi|    63863.

In [89]:
mydata2 = mydata2.drop("City").withColumnRenamed("clean_city", "City").drop("Salary","clean_salary").withColumnRenamed("new_salary", "Salary")

In [90]:
mydata2.show()

+---+----------+----------+------+--------------------+----------+-----------+---------------+--------+
| id|first_name| last_name|gender|            JobTitle|  Latitude|  Longitude|           City|  Salary|
+---+----------+----------+------+--------------------+----------+-----------+---------------+--------+
|  1|   Melinde| Shilburne|Female| Assistant Professor|50.5774075| 16.4967184|      Nowa Ruda|57438.18|
|  2|  Kimberly|Von Welden|Female|       Programmer II|48.8231572|103.5218199|         Bulgan| 62846.6|
|  4|   Shannon| O'Griffin|  Male|Budget/Accounting...|44.5047212| 38.1300171|  Divnomorskoye|61489.23|
|  5|  Sherwood|   Macieja|  Male|            VP Sales|      null| 37.6489954|      Mytishchi|63863.09|
|  6|     Maris|      Folk|Female|      Civil Engineer|53.4266145| -6.1644997|Kinsealy-Drinan|30101.16|
|  8|   Goddart|     Flear|  Male|Desktop Support T...|45.1905186|  0.7423124|      Trélissac|46116.36|
|  9|      Roth|O'Cannavan|  Male|VP Product Manage...| 32.02793

In [107]:
import numpy as np
median = np.median(mydata2.filter(mydata2.Latitude.isNotNull()).select(mydata2.Latitude.cast("float")).collect())

In [106]:
median2 = mydata2.withColumn("Latitude", mydata.Latitude.cast("float")).approxQuantile("Latitude", [0.5], 0.0)[0]

In [108]:
median
median2

31.93397331237793

In [115]:
mydata2.groupBy("City").agg(avg("Salary").alias("avg_city_sal")).sort(col("avg_city_sal").desc()).show()

+-----------------+------------+
|             City|avg_city_sal|
+-----------------+------------+
|        Mesopotam|    99948.28|
|       Zhongcheng|    99942.92|
|           Caxias|     99786.4|
|      Karangtawar|    99638.99|
|        Itabaiana|    99502.16|
|           Pasian|    99421.34|
|           Webuye|    99368.55|
|      Yuktae-dong|    99250.83|
|           Zinder|    99222.84|
|   Timiryazevskiy|    99142.94|
|        Sawahbaru|    99013.71|
|          Madimba|    98737.87|
|         Huangshi|    98690.34|
|          Gharyan|    98679.31|
|         Yŏnan-ŭp|    98628.61|
|     Wringinputih|    98603.82|
|Monte da Boavista|    98586.72|
|          Klukeng|    98439.49|
|         Murmashi|    98226.16|
|        Fox Creek|     98138.0|
+-----------------+------------+
only showing top 20 rows



In [123]:
df_avg_city_sal = mydata2.groupBy("City").agg(avg("Salary").alias("avg_city_sal"))

from pyspark.sql.window import Window
wndw_spec = Window.orderBy(df_avg_city_sal.avg_city_sal.desc())

df_avg_city_sal = df_avg_city_sal.withColumn("rn", row_number().over(wndw_spec))
df_avg_city_sal = df_avg_city_sal.filter(df_avg_city_sal.rn == 1).drop("rn")
df_avg_city_sal.show()

+---------+------------+
|     City|avg_city_sal|
+---------+------------+
|Mesopotam|    99948.28|
+---------+------------+

